# Initialization

Test notebook for the C-MAPPS benchmark. Get best parameters for each dataset using brute force search. 

First we import the necessary packages and create the global variables.

In [1]:
import math
import time
import numpy as np
import csv

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
import CMAPSAuxFunctions

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

from scipy.optimize import differential_evolution

Using TensorFlow backend.


# Define the model

In [2]:
K.clear_session()  #Clear the previous tensorflow graph

l2_lambda_regularization = 0.20
l1_lambda_regularization = 0.20

def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

# Get a new model

Function to get new model with new optimizer

In [3]:
def get_compiled_model(model_def, shape, model_type='ann'):

    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    if model_type=='ann':
        model = model_def(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    else:
        pass

    return model

models = {'shallow-20':RULmodel_SN_5}

# Perform exhaustive search

In [ ]:
#Perform exhaustive search to find the optimal parameters

#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'
num_features = len(selected_features)

window_size = 30
window_stride = 1
max_rul = 125
shape = num_features*window_size

#maxWindowSize = {'1':30, '2':20, '3':30, '4':18}
max_window_size = {'1':30, '2':20} #Do it only for datasets 1 and 2
total_time = {'1':0, '2':0, '3':0, '4':0}
results = {'1':0, '2':0, '3':0, '4':0}

lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

#Create necessary objects
dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)

model = get_compiled_model(models['shallow-20'], shape, model_type='ann')
tunable_model = SequenceTunableModelRegression('ModelRUL_SN_5', model, lib_type='keras', data_handler=dHandler_cmaps,
                                              epochs=20)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
tunable_model.data_scaler = min_max_scaler

for dataset_number in max_window_size:
    
    print("Results for dataset "+dataset_number)
    file = open("results/MLP/exhauxtive_search_"+dataset_number+".csv", "w")
    start_time = time.clock()
    
    tunable_model.data_handler.change_dataset(dataset_number)
    
    writer = csv.writer(file)
    verbose = 1
    
    for r in range(90, 141):   #Load max_rul first as it forces reloading the dataset from file
        
        verbose = 0
        tunable_model.data_handler.max_rul = r
        
        for w in range(15, max_window_size[dataset_number]+1):
        
            for s in range(1,11):
                
                print("Testing for w:{}, s:{}, r:{}".format(w, s, r))
                
                #Set data parameters
                tunable_model.data_handler.sequence_length = w
                tunable_model.data_handler.sequence_stride = s

                #Create and compile the models
                shape = num_features*w
                model = get_compiled_model(models['shallow-20'], shape, model_type='ann')

                #Add model to tunable model
                tunable_model.change_model('ModelRUL_SN', model, 'keras')
                                
                #Load the data
                tunable_model.load_data(unroll=True, verbose=verbose, cross_validation_ratio=0)
                
                #Train and evaluate
                tunable_model.train_model(learningRate_scheduler=lrate, verbose=0)
                tunable_model.evaluate_model(['rhs', 'rmse'], round=2)


                cScores = tunable_model.scores
                rmse = math.sqrt(cScores['score_1'])
                rmse2 = cScores['rmse']
                rhs = cScores['rhs']
                time = tunable_model.train_time
                
                row = [w, s, r, rmse, rhs]
                writer.writerow(row)
                
                #msgStr = "The model variables are " + str(x) + "\tThe scores are: [RMSE:{:.4f}, RHS:{:.4f}]\n".format(rmse, rhs)
                #file.write(msgStr)
                
                
    end_time = time.clock()
    file.close()
    totalTime[dataset_number] = end_time - start_time


Results for dataset 1
Testing for w:15, s:1, r:90
Loading data from file and computing dataframes
100/100 [==============================] - 0s 279us/step
Testing for w:15, s:2, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 0s 349us/step
Testing for w:15, s:3, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 0s 419us/step
Testing for w:15, s:4, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 0s 499us/step
Testing for w:15, s:5, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 0s 578us/step
Testing for w:15, s:6, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 0s 648us/step
Testing for w:15, s:7, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 0s 748us/step
Testing for w:15, s:8, r:90
Loading data from m

100/100 [==============================] - 1s 6ms/step
Testing for w:21, s:4, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 1s 6ms/step
Testing for w:21, s:5, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 1s 6ms/step
Testing for w:21, s:6, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 1s 6ms/step
Testing for w:21, s:7, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 1s 6ms/step
Testing for w:21, s:8, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 1s 6ms/step
Testing for w:21, s:9, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 1s 6ms/step
Testing for w:21, s:10, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 1s 7ms/step
Testing for w:22, s:1, r:90

100/100 [==============================] - 1s 13ms/step
Testing for w:27, s:6, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 1s 13ms/step
Testing for w:27, s:7, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 1s 13ms/step
Testing for w:27, s:8, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 1s 13ms/step
Testing for w:27, s:9, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 1s 13ms/step
Testing for w:27, s:10, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 1s 14ms/step
Testing for w:28, s:1, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 1s 13ms/step
Testing for w:28, s:2, r:90
Loading data from memory without recomputing df
100/100 [==============================] - 1s 14ms/step
Testing for w:28, s

100/100 [==============================] - 2s 20ms/step
Testing for w:17, s:8, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 2s 20ms/step
Testing for w:17, s:9, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 2s 20ms/step
Testing for w:17, s:10, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 2s 20ms/step
Testing for w:18, s:1, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 2s 21ms/step
Testing for w:18, s:2, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 2s 21ms/step
Testing for w:18, s:3, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 2s 21ms/step
Testing for w:18, s:4, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 2s 21ms/step
Testing for w:18, s

100/100 [==============================] - 3s 29ms/step
Testing for w:23, s:10, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 3s 29ms/step
Testing for w:24, s:1, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 3s 29ms/step
Testing for w:24, s:2, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 3s 30ms/step
Testing for w:24, s:3, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 3s 30ms/step
Testing for w:24, s:4, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 3s 30ms/step
Testing for w:24, s:5, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 3s 30ms/step
Testing for w:24, s:6, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 3s 30ms/step
Testing for w:24, s

100/100 [==============================] - 4s 38ms/step
Testing for w:30, s:2, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 4s 38ms/step
Testing for w:30, s:3, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 4s 39ms/step
Testing for w:30, s:4, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 4s 39ms/step
Testing for w:30, s:5, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 4s 39ms/step
Testing for w:30, s:6, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 4s 39ms/step
Testing for w:30, s:7, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 4s 39ms/step
Testing for w:30, s:8, r:91
Loading data from memory without recomputing df
100/100 [==============================] - 4s 40ms/step
Testing for w:30, s:

100/100 [==============================] - 5s 49ms/step
Testing for w:20, s:4, r:92
Loading data from memory without recomputing df
100/100 [==============================] - 5s 49ms/step
Testing for w:20, s:5, r:92
Loading data from memory without recomputing df
100/100 [==============================] - 5s 49ms/step
Testing for w:20, s:6, r:92
Loading data from memory without recomputing df
100/100 [==============================] - 5s 50ms/step
Testing for w:20, s:7, r:92
Loading data from memory without recomputing df
100/100 [==============================] - 5s 50ms/step
Testing for w:20, s:8, r:92
Loading data from memory without recomputing df
100/100 [==============================] - 5s 50ms/step
Testing for w:20, s:9, r:92
Loading data from memory without recomputing df
100/100 [==============================] - 5s 50ms/step
Testing for w:20, s:10, r:92
Loading data from memory without recomputing df
100/100 [==============================] - 5s 50ms/step
Testing for w:21, s

100/100 [==============================] - 6s 61ms/step
Testing for w:26, s:6, r:92
Loading data from memory without recomputing df
100/100 [==============================] - 6s 62ms/step
Testing for w:26, s:7, r:92
Loading data from memory without recomputing df
100/100 [==============================] - 6s 61ms/step
Testing for w:26, s:8, r:92
Loading data from memory without recomputing df
100/100 [==============================] - 6s 61ms/step
Testing for w:26, s:9, r:92
Loading data from memory without recomputing df
100/100 [==============================] - 6s 61ms/step
Testing for w:26, s:10, r:92
Loading data from memory without recomputing df
100/100 [==============================] - 6s 61ms/step
Testing for w:27, s:1, r:92
Loading data from memory without recomputing df
100/100 [==============================] - 6s 62ms/step
Testing for w:27, s:2, r:92
Loading data from memory without recomputing df
100/100 [==============================] - 6s 62ms/step
Testing for w:27, s

100/100 [==============================] - 8s 75ms/step
Testing for w:16, s:8, r:93
Loading data from memory without recomputing df
100/100 [==============================] - 8s 77ms/step
Testing for w:16, s:9, r:93
Loading data from memory without recomputing df
100/100 [==============================] - 8s 76ms/step
Testing for w:16, s:10, r:93
Loading data from memory without recomputing df
100/100 [==============================] - 8s 76ms/step
Testing for w:17, s:1, r:93
Loading data from memory without recomputing df
100/100 [==============================] - 8s 77ms/step
Testing for w:17, s:2, r:93
Loading data from memory without recomputing df
100/100 [==============================] - 8s 77ms/step
Testing for w:17, s:3, r:93
Loading data from memory without recomputing df
100/100 [==============================] - 8s 78ms/step
Testing for w:17, s:4, r:93
Loading data from memory without recomputing df
100/100 [==============================] - 8s 78ms/step
Testing for w:17, s

100/100 [==============================] - 9s 90ms/step
Testing for w:22, s:10, r:93
Loading data from memory without recomputing df
100/100 [==============================] - 9s 90ms/step
Testing for w:23, s:1, r:93
Loading data from memory without recomputing df
100/100 [==============================] - 9s 90ms/step
Testing for w:23, s:2, r:93
Loading data from memory without recomputing df
100/100 [==============================] - 9s 90ms/step
Testing for w:23, s:3, r:93
Loading data from memory without recomputing df
100/100 [==============================] - 9s 91ms/step
Testing for w:23, s:4, r:93
Loading data from memory without recomputing df
100/100 [==============================] - 9s 94ms/step
Testing for w:23, s:5, r:93
Loading data from memory without recomputing df
100/100 [==============================] - 9s 92ms/step
Testing for w:23, s:6, r:93
Loading data from memory without recomputing df
100/100 [==============================] - 9s 92ms/step
Testing for w:23, s

In [9]:
print("Total time {}".format(totalTime))
print(results)

Total time {'4': 0, '1': 18118.907471727885, '2': 15901.300738888152, '3': 0}
59.27322450422804
